In [16]:
import os
os.getcwd()

'/workspace'

In [21]:
import os
import shutil
import pandas as pd

In [22]:
generated_data_folder = 'generated_data'
train_src_folder = os.path.join(generated_data_folder, 'sah_filtered')
val_src_folder = os.path.join(generated_data_folder, 'val')
train_labels_file = os.path.join(train_src_folder, 'labels.csv')
val_labels_file = os.path.join(val_src_folder, 'labels.csv')

target_data_folder = 'data/sah'
train_target_images = os.path.join(target_data_folder, 'train', 'images')
train_target_labels = os.path.join(target_data_folder, 'train', 'labels')
val_target_images = os.path.join(target_data_folder, 'val', 'images')
val_target_labels = os.path.join(target_data_folder, 'val', 'labels')

os.makedirs(train_target_images, exist_ok=True)
os.makedirs(train_target_labels, exist_ok=True)
os.makedirs(val_target_images, exist_ok=True)
os.makedirs(val_target_labels, exist_ok=True)

In [23]:
def prepare_split(src_images_folder, src_labels_file, target_images_folder, target_labels_folder):
    df = pd.read_csv(src_labels_file)
    
    df['filename'] = df['filename'].apply(lambda x: os.path.join('images', x))
    
    target_labels_file = os.path.join(target_labels_folder, 'labels.csv')
    df.to_csv(target_labels_file, index=False, encoding='utf-8')
    print(f"Файл меток сохранён: {target_labels_file}")
    
    for idx, row in df.iterrows():
        src_img_path = os.path.join(src_images_folder, row['filename'].split('images/')[1])
        target_img_path = os.path.join(target_images_folder, row['filename'].split('images/')[1])
        if not os.path.exists(src_img_path):
            print(f"Изображение не найдено: {src_img_path}")
            continue
        shutil.copy(src_img_path, target_img_path)
        if (idx + 1) % 1000 == 0:
            print(f"Скопировано {idx + 1} изображений...")
    
    print(f"Копирование изображений из {src_images_folder} завершено.")

In [24]:
prepare_split(train_src_folder, train_labels_file, train_target_images, train_target_labels)

Файл меток сохранён: data/sah/train/labels/labels.csv
Скопировано 1000 изображений...
Скопировано 2000 изображений...
Скопировано 3000 изображений...
Скопировано 4000 изображений...
Скопировано 5000 изображений...
Скопировано 6000 изображений...
Скопировано 7000 изображений...
Скопировано 8000 изображений...
Скопировано 9000 изображений...
Скопировано 10000 изображений...
Скопировано 11000 изображений...
Скопировано 12000 изображений...
Скопировано 13000 изображений...
Скопировано 14000 изображений...
Скопировано 15000 изображений...
Скопировано 16000 изображений...
Скопировано 17000 изображений...
Копирование изображений из generated_data/sah_filtered завершено.


In [25]:
print("Подготовка валидационного набора...")
prepare_split(val_src_folder, val_labels_file, val_target_images, val_target_labels)

Подготовка валидационного набора...
Файл меток сохранён: data/sah/val/labels/labels.csv
Скопировано 1000 изображений...
Скопировано 2000 изображений...
Скопировано 3000 изображений...
Скопировано 4000 изображений...
Копирование изображений из generated_data/val завершено.


In [26]:
%cd deep-text-recognition-benchmark
os.getcwd()

/workspace/deep-text-recognition-benchmark


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/workspace/deep-text-recognition-benchmark'

In [43]:
!pip install lmdb fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 272.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114247 sha256=ca2b3e5662087b343a5a71e379d981be34687b1aa2071f6ae9d879e7ee49a52c
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [41]:
!CUDA_VISIBLE_DEVICES=0,1,2,3,4 python3 train.py --train_data /workspace/data/sah/train/images --valid_data /workspace/data/sah/val/images --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn --batch_size 64 --num_iter 300000 --exp_name sah_experiment

------ Use multi-GPU setting ------
if you stuck too long time with multi-GPU setting, try to set --workers 0
Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: /workspace/data/sah/train/images
opt.select_data: ['MJ', 'ST']
opt.batch_ratio: ['0.5', '0.5']
--------------------------------------------------------------------------------
dataset_root:    /workspace/data/sah/train/images	 dataset: MJ
Traceback (most recent call last):
  File "/workspace/deep-text-recognition-benchmark/train.py", line 317, in <module>
    train(opt)
  File "/workspace/deep-text-recognition-benchmark/train.py", line 31, in train
    train_dataset = Batch_Balanced_Dataset(opt)
  File "/workspace/deep-text-recognition-benchmark/dataset.py", line 42, in __init__
    _dataset, _dataset_log = hierarchical_dataset(root=opt.train_d

In [44]:
!python3 create_lmdb_dataset.py --inputPath /workspace/data/sah/train/images --gtFile /workspace/data/sah/train/labels/labels.csv --outputPath /workspace/data/sah_lmdb/train

Traceback (most recent call last):
  File "/workspace/deep-text-recognition-benchmark/create_lmdb_dataset.py", line 87, in <module>
    fire.Fire(createDataset)
  File "/opt/conda/lib/python3.10/site-packages/fire/core.py", line 135, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/opt/conda/lib/python3.10/site-packages/fire/core.py", line 468, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/opt/conda/lib/python3.10/site-packages/fire/core.py", line 684, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
  File "/workspace/deep-text-recognition-benchmark/create_lmdb_dataset.py", line 47, in createDataset
    imagePath, label = datalist[i].strip('\n').split('\t')
ValueError: not enough values to unpack (expected 2, got 1)


In [49]:
import pandas as pd

# Загрузка исходного файла с метками
df = pd.read_csv('/workspace/data/sah/train/labels/labels.csv')
df1 = pd.read_csv('/workspace/data/sah/val/labels/labels.csv')

# Преобразование в нужный формат (замена запятой на табуляцию)
df.to_csv('/workspace/data/sah/train/labels/labels.txt', sep='\t', index=False, header=False)
df1.to_csv('/workspace/data/sah/val/labels/labels.txt', sep='\t', index=False, header=False)

In [48]:
!python3 create_lmdb_dataset.py --inputPath /workspace/data/sah/train --gtFile /workspace/data/sah/train/labels/labels.txt --outputPath /workspace/data/sah_lmdb/train

Written 1000 / 17405
Written 2000 / 17405
Written 3000 / 17405
Written 4000 / 17405
Written 5000 / 17405
Written 6000 / 17405
Written 7000 / 17405
Written 8000 / 17405
Written 9000 / 17405
Written 10000 / 17405
Written 11000 / 17405
Written 12000 / 17405
Written 13000 / 17405
Written 14000 / 17405
Written 15000 / 17405
Written 16000 / 17405
Written 17000 / 17405
Created dataset with 17405 samples


In [51]:
!python3 create_lmdb_dataset.py --inputPath /workspace/data/sah/val --gtFile /workspace/data/sah/val/labels/labels.txt --outputPath /workspace/data/sah_lmdb/val

Written 1000 / 4352
Written 2000 / 4352
Written 3000 / 4352
Written 4000 / 4352
Created dataset with 4352 samples


In [52]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 python3 train.py \
--train_data /workspace/data/sah_lmdb/train \
--valid_data /workspace/data/sah_lmdb/val \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--batch_size 64 --num_iter 300000 --exp_name sah_experiment

------ Use multi-GPU setting ------
if you stuck too long time with multi-GPU setting, try to set --workers 0
Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: /workspace/data/sah_lmdb/train
opt.select_data: ['MJ', 'ST']
opt.batch_ratio: ['0.5', '0.5']
--------------------------------------------------------------------------------
dataset_root:    /workspace/data/sah_lmdb/train	 dataset: MJ
Traceback (most recent call last):
  File "/workspace/deep-text-recognition-benchmark/train.py", line 317, in <module>
    train(opt)
  File "/workspace/deep-text-recognition-benchmark/train.py", line 31, in train
    train_dataset = Batch_Balanced_Dataset(opt)
  File "/workspace/deep-text-recognition-benchmark/dataset.py", line 42, in __init__
    _dataset, _dataset_log = hierarchical_dataset(root=opt.train_data,

In [53]:
!ls /workspace/data/sah_lmdb/train

data.mdb  lock.mdb


In [63]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 python3 deep-text-recognition-benchmark/train.py \
--train_data data/sah_lmdb/train \
--valid_data data/sah_lmdb/val \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--batch_size 64 --num_iter 300000 --exp_name sah_experiment \
--select_data 'default' --batch_ratio 1

------ Use multi-GPU setting ------
if you stuck too long time with multi-GPU setting, try to set --workers 0
Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: data/sah_lmdb/train
opt.select_data: ['default']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    data/sah_lmdb/train	 dataset: default
Traceback (most recent call last):
  File "/workspace/deep-text-recognition-benchmark/train.py", line 317, in <module>
    train(opt)
  File "/workspace/deep-text-recognition-benchmark/train.py", line 31, in train
    train_dataset = Batch_Balanced_Dataset(opt)
  File "/workspace/deep-text-recognition-benchmark/dataset.py", line 42, in __init__
    _dataset, _dataset_log = hierarchical_dataset(root=opt.train_data, opt=opt, select_data=[sele

In [64]:
os.getcwd()

'/workspace'

In [65]:
!ls data/sah_lmdb/train
!ls data/sah_lmdb/val

data.mdb  lock.mdb
data.mdb  lock.mdb


In [67]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 python3 deep-text-recognition-benchmark/train.py \
--train_data data/sah_lmdb/train \
--valid_data data/sah_lmdb/val \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--batch_size 64 --num_iter 300000 --exp_name sah_experiment

------ Use multi-GPU setting ------
if you stuck too long time with multi-GPU setting, try to set --workers 0
Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: data/sah_lmdb/train
opt.select_data: ['MJ', 'ST']
opt.batch_ratio: ['0.5', '0.5']
--------------------------------------------------------------------------------
dataset_root:    data/sah_lmdb/train	 dataset: MJ
Traceback (most recent call last):
  File "/workspace/deep-text-recognition-benchmark/train.py", line 317, in <module>
    train(opt)
  File "/workspace/deep-text-recognition-benchmark/train.py", line 31, in train
    train_dataset = Batch_Balanced_Dataset(opt)
  File "/workspace/deep-text-recognition-benchmark/dataset.py", line 42, in __init__
    _dataset, _dataset_log = hierarchical_dataset(root=opt.train_data, opt=opt, select_data=